In [1]:
import pandas as pd
import numpy as np
import os
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer.autoguide import AutoDiagonalNormal
import seaborn as sns
import matplotlib.pyplot as plt
import math



smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.9.0')

We start with a simple model: choose a coin. Ground truth is the probability of heads is drawn from a beta(10,10). Then we do variational inference on this probability f by setting the variational family beta.

In [2]:
def model(data):
    # define the hyperparameters that control the Beta prior

    f1 = pyro.sample("latent_fairness_1", dist.Beta(10, 10))
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the Bernoulli
        # likelihood Bernoulli(f)
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f1), obs=data[i])

def guide_custome(model):

    # register the two variational parameters with Pyro.
    alpha_q1 = pyro.param("alpha_q1", torch.tensor(15.0),
                         constraint=dist.constraints.positive)
    beta_q1 = pyro.param("beta_q1", torch.tensor(15.0),
                        constraint=dist.constraints.positive)

    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness_1", dist.Beta(alpha_q1, beta_q1))


# setup the optimizer
adam_params = {"lr": 0.0005}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide_custome, optimizer, loss=Trace_ELBO())

data = []
for _ in range(6):
    data.append(torch.tensor(1.0))
for _ in range(4):
    data.append(torch.tensor(0.0))


# do gradient steps
n_steps = 2000
losses = []

for step in range(n_steps):
    loss = svi.step(data)
    losses.append(loss)
    if step % 100 == 0:
        print(f"Step {step}: Loss = {loss}")


alpha_q1 = pyro.param("alpha_q1").item()
beta_q1 = pyro.param("beta_q1").item()

inferred_mean = alpha_q1 / (alpha_q1 + beta_q1)
# compute inferred standard deviation
factor = beta_q1 / (alpha_q1 * (1.0 + alpha_q1 + beta_q1))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nBased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))

Step 0: Loss = 7.213503122329712
Step 100: Loss = 7.092877984046936
Step 200: Loss = 7.196681976318359
Step 300: Loss = 7.0459043979644775
Step 400: Loss = 7.070078015327454
Step 500: Loss = 7.090681076049805
Step 600: Loss = 7.023242712020874
Step 700: Loss = 7.077595472335815
Step 800: Loss = 7.107882022857666
Step 900: Loss = 6.973265290260315
Step 1000: Loss = 7.118948936462402
Step 1100: Loss = 6.958721160888672
Step 1200: Loss = 7.103104591369629
Step 1300: Loss = 7.0642160177230835
Step 1400: Loss = 7.059703230857849
Step 1500: Loss = 7.075753092765808
Step 1600: Loss = 7.032650589942932
Step 1700: Loss = 7.054135680198669
Step 1800: Loss = 7.046597838401794
Step 1900: Loss = 7.0928120613098145

Based on the data and our prior belief, the fairness of the coin is 0.535 +- 0.090


Now we use the autoguide function autoDiagonalNormal to see if ADVI can solve the problem.

In [3]:
auto_guide = AutoDiagonalNormal(model)
svi = SVI(model, auto_guide, optimizer, loss=Trace_ELBO())

n_steps = 2000
losses = []

for step in range(n_steps):
    loss = svi.step(data)
    losses.append(loss)
    if step % 100 == 0:
        print(f"Step {step}: Loss = {loss}")

for name, value in pyro.get_param_store().items():
    print(f"{name} = {value.detach().cpu().numpy()}")


Step 0: Loss = 6.1860562562942505
Step 100: Loss = 8.403323292732239
Step 200: Loss = 7.725732088088989
Step 300: Loss = 8.254777193069458
Step 400: Loss = 8.21003532409668
Step 500: Loss = 8.004858374595642
Step 600: Loss = 6.89823317527771
Step 700: Loss = 8.006877779960632
Step 800: Loss = 6.200948596000671
Step 900: Loss = 7.560607552528381
Step 1000: Loss = 7.92170774936676
Step 1100: Loss = 7.88515031337738
Step 1200: Loss = 7.842462420463562
Step 1300: Loss = 7.314276099205017
Step 1400: Loss = 7.757480263710022
Step 1500: Loss = 7.704437255859375
Step 1600: Loss = 7.552847504615784
Step 1700: Loss = 7.633719563484192
Step 1800: Loss = 7.615230202674866
Step 1900: Loss = 7.443209886550903
alpha_q1 = 16.04521942138672
beta_q1 = 13.945406913757324
AutoDiagonalNormal.loc = [0.14202186]
AutoDiagonalNormal.scale = [0.22710498]


Try the mannual implementation.

We see the inferred loc and scale is far from true. What's worse is that if we change the model to include two parameters. 

In [5]:
def model(data):
    # define the hyperparameters that control the Beta prior

    # sample f from the Beta prior
    f1 = pyro.sample("latent_fairness_1", dist.Beta(10, 10))
    f2 = pyro.sample("latent_fairness_2", dist.Beta(15, 10))
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the Bernoulli
        # likelihood Bernoulli(f)
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f1*f2), obs=data[i])

pyro.render_model(model, model_args=(data), render_distributions=True, render_params=True)


AssertionError: 

In [23]:


def guide_custome(model):

    # register the two variational parameters with Pyro.
    alpha_q1 = pyro.param("alpha_q1", torch.tensor(15.0),
                         constraint=dist.constraints.positive)
    beta_q1 = pyro.param("beta_q1", torch.tensor(15.0),
                        constraint=dist.constraints.positive)
    alpha_q2 = pyro.param("alpha_q2", torch.tensor(15.0),
                         constraint=dist.constraints.positive)
    beta_q2 = pyro.param("beta_q2", torch.tensor(15.0),
                        constraint=dist.constraints.positive)
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness_1", dist.Beta(alpha_q1, beta_q1))
    pyro.sample("latent_fairness_2", dist.Beta(alpha_q2, beta_q2))

# setup the optimizer
adam_params = {"lr": 0.0005}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide_custome, optimizer, loss=Trace_ELBO())

data = []
for _ in range(6):
    data.append(torch.tensor(1.0))
for _ in range(4):
    data.append(torch.tensor(0.0))


# do gradient steps
n_steps = 2000
losses = []

for step in range(n_steps):
    loss = svi.step(data)
    losses.append(loss)
    if step % 100 == 0:
        print(f"Step {step}: Loss = {loss}")


alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nBased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))

Step 0: Loss = 8.728413105010986
Step 100: Loss = 8.319986701011658
Step 200: Loss = 9.983762741088867
Step 300: Loss = 9.776553630828857
Step 400: Loss = 5.533115863800049
Step 500: Loss = 8.818577289581299
Step 600: Loss = 9.619333267211914
Step 700: Loss = 8.925556898117065
Step 800: Loss = 8.452331185340881
Step 900: Loss = 7.522561430931091
Step 1000: Loss = 8.288254022598267
Step 1100: Loss = 8.806099653244019
Step 1200: Loss = 8.526350259780884
Step 1300: Loss = 8.105169773101807
Step 1400: Loss = 8.950329542160034
Step 1500: Loss = 8.330858945846558
Step 1600: Loss = 8.433443069458008
Step 1700: Loss = 8.39637565612793
Step 1800: Loss = 8.413408160209656
Step 1900: Loss = 8.325928211212158

Based on the data and our prior belief, the fairness of the coin is 0.530 +- 0.090


In [25]:
for name, value in pyro.get_param_store().items():
    print(f"{name} = {value.detach().cpu().numpy()}")

alpha_q = 15.89972972869873
beta_q = 14.127716064453125
AutoDiagonalNormal.loc = [0.14392696]
AutoDiagonalNormal.scale = [0.36030772]
alpha_q1 = 17.080364227294922
beta_q1 = 12.374051094055176
alpha_q2 = 20.10841178894043
beta_q2 = 10.899086952209473
